# VacationPy

## Starter Code to Import Libraries and Load the Weather and Coordinates Data¶

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
import matplotlib.pyplot as plt
# Import API key
from api_keys import geoapify_key

In [3]:
# Define the path to your CSV file
city_data_path = r"C:\Users\Mimy\OneDrive\Desktop\python-api-challenge\python-api-challenge\cities.csv"

# Load the CSV file into a DataFrame

city_data_df = pd.read_csv(city_data_path)
# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.43,73,71,3.61,PN,1731429380
1,1,jinghong,21.9910,100.7341,18.02,97,0,1.49,CN,1731429381
2,2,taulaga,-11.0553,-171.0883,28.22,69,99,6.51,AS,1731429383
3,3,bethel,41.3712,-73.4140,10.68,52,40,6.17,US,1731429384
4,4,gaspe,48.8334,-64.4819,3.12,87,75,3.09,CA,1731429385


## Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [7]:
# Plot cities on the holoview plot

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria, I like hot environments! 

narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] <31) & (city_data_df["Max Temp"] > 22) \
    & (city_data_df["Wind Speed"] < 2) \
    & (city_data_df["Cloudiness"] == 0),:
]

# Drop null values
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,east london,-33.0153,27.9116,23.52,73,0,1.54,ZA,1731429447
79,79,cabo san lucas,22.8909,-109.9124,28.20,56,0,1.54,MX,1731429470
203,203,mansa,29.9833,75.3833,22.37,22,0,1.39,IN,1731429613
250,250,sur,22.5667,59.5289,27.28,58,0,0.26,OM,1731429667


## Step 3: Create a new DataFrame called hotel_df

In [11]:
# using a copy function to create hotel_df
hotel_df = narrowed_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()  # Explicitly create a full copy
# creating hotel name column with empty values
hotel_df['Hotel Name'] = ''
# displaying ten sample row
sample_hotel_df = hotel_df.sample(10, replace = True)
sample_hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
59,east london,ZA,-33.0153,27.9116,73,
79,cabo san lucas,MX,22.8909,-109.9124,56,
250,sur,OM,22.5667,59.5289,58,
79,cabo san lucas,MX,22.8909,-109.9124,56,
203,mansa,IN,29.9833,75.3833,22,
79,cabo san lucas,MX,22.8909,-109.9124,56,
59,east london,ZA,-33.0153,27.9116,73,
59,east london,ZA,-33.0153,27.9116,73,
203,mansa,IN,29.9833,75.3833,22,
203,mansa,IN,29.9833,75.3833,22,


## Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "categories": "accommodation.hotel",
        "limit": 20,  # Only get the 20 results
        "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address =  requests.get(base_url, params=params)
    print(name_address.status_code)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
200
east london - nearest hotel: No hotel found
200
cabo san lucas - nearest hotel: Comfort Rooms
200
mansa - nearest hotel: No hotel found
200
sur - nearest hotel: Sur Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
59,east london,ZA,-33.0153,27.9116,73,No hotel found
79,cabo san lucas,MX,22.8909,-109.9124,56,Comfort Rooms
203,mansa,IN,29.9833,75.3833,22,No hotel found
250,sur,OM,22.5667,59.5289,58,Sur Hotel


## Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map

In [14]:
# Plot holoview and show hotels on the map

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)